# 特徴ベースの知識蒸留フレームワーク

## 🧪 仮説
生徒モデルの処理の流れを教師モデルに似せることで、生徒モデルは教師モデルの性能に近づくことができる。

---

## 🔍 特徴

- 中間層の特徴表現を活用
- CKA（Centered Kernel Alignment）による層間類似度の評価
- 構造的類似性の模倣：処理の流れそのものを模倣する
  - 例：教師モデルが序盤で学習する特徴は、生徒モデルも序盤で学習するように誘導する
- 層数の違いを吸収可能：層数が異なるモデル間でも蒸留が可能

---

## ⚙️ 実装手順

1. 教師モデルの準備  
   - 多くの場合、事前学習済みモデルを使用

2. 教師モデルの層間CKA計算  
   - 各層の出力に対してCKAを計算し、類似度を評価

3. 層のグルーピング  
   - 教師モデルの層を CKA に基づいて n グループに分割  
   - グループは順番を保持し、隣接する層のみ同じグループに所属可能

4. 生徒モデルの分割  
   - 生徒モデルも同様に n グループに分割（層数が異なっていても均等に分割）

5. グループ対応付け  
   - 教師モデルと生徒モデルの各グループを順番に対応させ、n × n の対応関係を構築

---

## 📉 損失関数設計

- 毎バッチで損失を計算
- 生徒モデルと教師モデルの各層の出力に対して CKA を計算
- L_s × L_t の CKA 類似度マトリクスを構築  
  - L_s：生徒モデルの層数  
  - L_t：教師モデルの層数

- 対角成分に対応する n グループ（G₁, G₂, ..., Gₙ）に注目
- 各グループ Gᵢ の代表値（平均など）を算出 → CKA_Gᵢ
- 損失として 1 - CKA_Gᵢ を計算
- 全グループの損失を統合（加重平均、合計など）して最終損失とする

---

## ✨ メリット

- 層数の違いを吸収しながら、構造的な知識を効果的に蒸留可能
- CKAにより、単なる出力の一致ではなく特徴空間の類似性を重視
- 処理の流れを模倣することで、より深い知識の転移が可能


# 実装
train_student.pyをベースに実装

In [ ]:
from __future__ import print_function

import os
import re
import argparse
import time

import numpy
import torch
import torch.optim as optim
import torch.multiprocessing as mp
import torch.distributed as dist
import torch.nn as nn
import torch.backends.cudnn as cudnn
# import tensorboard_logger as tb_logger
# 変更後
from torch.utils.tensorboard import SummaryWriter

from models import model_dict
from models.util import ConvReg, SelfA, SRRL, SimKD

from dataset.cifar10 import get_cifar10_dataloaders, get_cifar10_dataloaders_sample
from dataset.cifar100 import get_cifar100_dataloaders, get_cifar100_dataloaders_sample
from dataset.imagenet import get_imagenet_dataloader,  get_dataloader_sample
from dataset.cinic10 import get_cinic10_dataloaders, get_cinic10_dataloaders_sample
# from dataset.FashionMNIST import get_FashionMNIST_dataloaders, get_FashionMNIST_dataloaders_sample
# from dataset.imagenet_dali import get_dali_data_loader

from helper.loops import train_distill as train, validate_vanilla, validate_distill
from helper.util import save_dict_to_json, reduce_tensor, adjust_learning_rate

from crd.criterion import CRDLoss
from distiller_zoo import DistillKL, HintLoss, Attention, Similarity, VIDLoss, SemCKDLoss

split_symbol = '~' if os.name == 'nt' else ':'

In [ ]:
def get_teacher_name(model_path):
    """parse teacher name"""
    directory = model_path.split('/')[-2]
    pattern = ''.join(['S', split_symbol, '(.+)', '_T', split_symbol])
    name_match = re.match(pattern, directory)
    if name_match:
        return name_match[1]
    segments = directory.split('_')
    if segments[0] == 'wrn':
        return segments[0] + '_' + segments[1] + '_' + segments[2]
    return segments[0]

In [ ]:
def load_teacher(model_path, n_cls, gpu=None, opt=None):
    print('==> loading teacher model')
    model_t = get_teacher_name(model_path)
    model = model_dict[model_t](num_classes=n_cls)
    map_location = None if gpu is None else {'cuda:0': 'cuda:%d' % (gpu if opt.multiprocessing_distributed else 0)}
    model.load_state_dict(torch.load(model_path, map_location=map_location)['model'])
    print('==> done')
    return model

In [ ]:
def parse_option():

    parser = argparse.ArgumentParser('argument for training')
    
    # basic
    parser.add_argument('--print_freq', type=int, default=200, help='print frequency')
    parser.add_argument('--batch_size', type=int, default=64, help='batch_size')
    parser.add_argument('--num_workers', type=int, default=8, help='num of workers to use')
    parser.add_argument('--epochs', type=int, default=240, help='number of training epochs')
    parser.add_argument('--gpu_id', type=str, default='0', help='id(s) for CUDA_VISIBLE_DEVICES')

    # optimization
    parser.add_argument('--learning_rate', type=float, default=0.05, help='learning rate')
    parser.add_argument('--lr_decay_epochs', type=str, default='150,180,210', help='where to decay lr, can be a list')
    parser.add_argument('--lr_decay_rate', type=float, default=0.1, help='decay rate for learning rate')
    parser.add_argument('--weight_decay', type=float, default=5e-4, help='weight decay')
    parser.add_argument('--momentum', type=float, default=0.9, help='momentum')

    # dataset and model
    parser.add_argument('--dataset', type=str, default='cifar10', choices=['cifar10', 'cifar100', 'imagenet', 'cinic10'], help='dataset')
    parser.add_argument('--model_s', type=str, default='resnet8x4')
    parser.add_argument('--path_t', type=str, default=None, help='teacher model snapshot')

    # distillation
    parser.add_argument('--trial', type=str, default='1', help='trial id')
    parser.add_argument('--kd_T', type=float, default=4, help='temperature for KD distillation')
    parser.add_argument('--distill', type=str, default='kd', choices=['kd', 'hint', 'attention', 'similarity', 'vid',
                                                                      'crd', 'semckd','srrl', 'simkd'])
    parser.add_argument('-c', '--cls', type=float, default=1.0, help='weight for classification')
    parser.add_argument('-d', '--div', type=float, default=1.0, help='weight balance for KD')
    parser.add_argument('-b', '--beta', type=float, default=0.0, help='weight balance for other losses')
    parser.add_argument('-f', '--factor', type=int, default=2, help='factor size of SimKD')
    parser.add_argument('-s', '--soft', type=float, default=1.0, help='attention scale of SemCKD')

    # hint layer
    parser.add_argument('--hint_layer', default=1, type=int, choices=[0, 1, 2, 3, 4])

    # NCE distillation
    parser.add_argument('--feat_dim', default=128, type=int, help='feature dimension')
    parser.add_argument('--mode', default='exact', type=str, choices=['exact', 'relax'])
    parser.add_argument('--nce_k', default=16384, type=int, help='number of negative samples for NCE')
    parser.add_argument('--nce_t', default=0.07, type=float, help='temperature parameter for softmax')
    parser.add_argument('--nce_m', default=0.5, type=float, help='momentum for non-parametric updates')

    # multiprocessing
    parser.add_argument('--dali', type=str, choices=['cpu', 'gpu'], default=None)
    parser.add_argument('--multiprocessing-distributed', action='store_true',
                    help='Use multi-processing distributed training to launch '
                         'N processes per node, which has N GPUs. This is the '
                         'fastest way to use PyTorch for either single node or '
                         'multi node data parallel training')
    parser.add_argument('--dist-url', default='tcp://127.0.0.1:23451', type=str,
                    help='url used to set up distributed training')
    parser.add_argument('--deterministic', action='store_true', help='Make results reproducible')
    parser.add_argument('--skip-validation', action='store_true', help='Skip validation of teacher')
    
    opt = parser.parse_args()

    # set different learning rates for these MobileNet/ShuffleNet models
    if opt.model_s in ['MobileNetV2', 'MobileNetV2_1_0', 'ShuffleV1', 'ShuffleV2', 'ShuffleV2_1_5']:
        opt.learning_rate = 0.01

    # set the path of model and tensorboard
    opt.model_path = './save/students/models'
    opt.tb_path = './save/students/tensorboard'

    iterations = opt.lr_decay_epochs.split(',')
    opt.lr_decay_epochs = list([])
    for it in iterations:
        opt.lr_decay_epochs.append(int(it))

    opt.model_t = get_teacher_name(opt.path_t)

    model_name_template = split_symbol.join(['S', '{}_T', '{}_{}_{}_r', '{}_a', '{}_b', '{}_{}'])
    opt.model_name = model_name_template.format(opt.model_s, opt.model_t, opt.dataset, opt.distill,
                                                opt.cls, opt.div, opt.beta, opt.trial)

    if opt.dali is not None:
        opt.model_name += '_dali:' + opt.dali

    opt.tb_folder = os.path.join(opt.tb_path, opt.model_name)
    if not os.path.isdir(opt.tb_folder):
        os.makedirs(opt.tb_folder)

    opt.save_folder = os.path.join(opt.model_path, opt.model_name)
    if not os.path.isdir(opt.save_folder):
        os.makedirs(opt.save_folder)
    
    return opt

### 1. 教師モデルの準備
既存のモデルを使うこともできる

In [ ]:
# import subprocess

# cmd = [
#     "python", "train_teacher.py",
#     "--dataset", "cinic10",
#     "--epochs", "240",
#     "--trial", "0",
#     "--model", "vgg8"
# ]

# subprocess.run(cmd)

### 2. 教師モデルの層間CKA計算  
   - 各層の出力に対してCKAを計算し、類似度を評価


### 3. 層のグルーピング  
   - 教師モデルの層を CKA に基づいて n グループに分割  
   - グループは順番を保持し、隣接する層のみ同じグループに所属可能

### 4. 生徒モデルの分割  
   - 生徒モデルも同様に n グループに分割（層数が異なっていても均等に分割）

### 5. グループ対応付け  
   - 教師モデルと生徒モデルの各グループを順番に対応させ、n × n の対応関係を構築